In [2]:
#__________________ Reading the data __________________#
with open('./data/MapData/ethnicity_df.pickle', 'rb') as fp:
    ethnicity_df = pickle.load(fp)

with open('./data/MapData/geoid_normalising.pickle', 'rb') as fp:
    geoid_normalising = pickle.load(fp)
    
with open('./data/chicago_inspections_ethnicity.pickle', 'rb') as fp:
    chicago_inspections = pickle.load(fp)
    
with open('./data/businessesChicago.pkl', 'rb') as fp:
    businesses = pickle.load(fp)
    
with open('./data/zip_ethnicity.pkl', 'rb') as fp:
    zip_ethnicity = pickle.load(fp)
    
tract = pd.read_excel('./data/ZIP_TRACT_122017.xlsx')
    
zipcode_path = './data/zipCodesBoundaries.geojson'

In [71]:
#__________________ Migrating the ethnicity dataset towards a zipcode structure __________________#
# Already saved as ./data/chicago_inspections_ethnicity.pickle

majority_ethnicity = ethnicity_df[ethnicity_df['lntitle'] != 'Total']
majority_ethnicity = majority_ethnicity[majority_ethnicity['lntitle'] != 'Not Hispanic or Latino']
majority_ethnicity['geoid_match'] = majority_ethnicity['geoid_match'].astype(np.int64)

chicago_inspections["majority_ethnicity"] = np.zeros(len(chicago_inspections)) 
nanCounters = 0
for i in range(len(chicago_inspections)):
    try:
        zipp = chicago_inspections.iloc[i]["zip"] # Get zip of that instance
        tract_linked = tract[tract['zip'] == int(zipp)] # Get all tract numbers linked to the zip
        tract_linked = tract_linked['tract'].unique()
        temp = majority_ethnicity[majority_ethnicity['geoid_match'].isin(tract_linked)] # retrieve all the statistics linked to those tracks
        temp = temp.groupby('lntitle').sum().reset_index()
        majority = temp.iloc[np.argmax(temp["CIT_EST"])]['lntitle']
        chicago_inspections["majority_ethnicity"].iloc[i] = majority
    except ValueError:
        nanCounters += 1
        print(nanCounters)
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [14]:
#__________________ Mapping Zip-Ethnicity __________________#
# Already saved as ./data/zip_ethnicity.pkl
zip_eth = chicago_inspections[chicago_inspections["majority_ethnicity"] != 0]
zip_ethnicity = {}
for zipcode in zip_eth['zip'].unique():
    zippi = str(int(zipcode))
    temp = zip_eth[zip_eth['zip'] == zipcode]
    zip_ethnicity[zippi] = temp['majority_ethnicity'].iloc[0]
zip_ethnicity = pd.DataFrame(zip_ethnicity, index=[0]).transpose().reset_index()
zip_ethnicity.columns = ['amount of zip areas dominated by x', 'ethnicity']

In [173]:
#__________________ Function to standardise the plots __________________#

def plot_bardata(data, x, y, title, hovertemplate, order={'ethnicity':ethnicities}):
    # Plotting the data
    fig = px.bar(
        data, x=x, y=y, 
        title=title,
        category_orders=order,
    )
    fig.update_traces(marker_color=data['color'], 
                      marker_line_color=data['color'],
                  marker_line_width=1.5, opacity=0.6, 
                  hovertemplate = hovertemplate)
    fig.update_layout(title_text=title)

    fig.show()

In [4]:
# Storing the majority ethnicity names
ethnicities = [
    'White Alone', #whites,    
    'Black or African American Alone', #baaa
    'Hispanic or Latino' #hol
]

In [17]:
#__________________ Plotting the amount of zipcodes dominated by each of the ethnicities __________________#
ethnicity_distribution = zip_ethnicity.groupby("ethnicity").nunique()[['amount of zip areas dominated by x']].reset_index()
ethnicity_distribution['color'] = ['dodgerblue', 'lightgreen', 'lightskyblue']

fig = px.bar(
    ethnicity_distribution, x='ethnicity', y='amount of zip areas dominated by x', 
    title="Distribution of Zip Areas Dominated by Ethnicity X",
    category_orders={'ethnicity':ethnicities},
)
fig.update_traces(marker_color=ethnicity_distribution['color'], marker_line_color=ethnicity_distribution['color'],
                  marker_line_width=1.5, opacity=0.6, 
                  hovertemplate = '%{y:.2f} zipcode areas are dominated by the ethnicity %{x}')
fig.update_layout(title_text='Number of Zip Code Areas Dominated by Each of the Ethnicities')

fig.show()

In [138]:
#---------------- Getting all restaurants ----------------#
## Waiting for Alfonso to get the google revs in place
chicago_inspections.head(1)



,inspection_id,dba_name,aka_name,license,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,location,new_facility_type,new_violations
0,2320977,todd appel bar and cocktail,todd appel bar and cocktail,2589615.0,grocery store,Risk 2 (Medium),5868 N LINCOLN AVE,chicago,IL,60659.0,2019-11-01T00:00:00.000,canvas,No Entry,NaN,41.98806,-87.703704,"{'latitude': '-87.70370445181302', 'longitude'...",grocery store,[]


In [12]:
# Base Map
chicago_map = folium.Map(location=[41.806167, -87.7], zoom_start=10)
lgd_txt = '<span style="color: {col};">{txt}</span>'

# Zipcode geometry
zipGeom = gpd.read_file("./data/zipCodesBoundaries.geojson")
zipGeom['zip'] = zipGeom['zip'].astype(int)
zipGeom['shape_area'] = zipGeom['shape_area'].astype(float)
chicago_inspections["zip"] = chicago_inspections['zip'].astype(int).astype(str)

# Plotting the white population
whites = list(zip_ethnicity[zip_ethnicity['ethnicity'] == 'White Alone']['zip'])
whites = zipGeom[zipGeom['zip'].isin(whites)]
folium.GeoJson(
    whites.to_json(),
    style_function=lambda feature: {
        'fillColor': "lightskyblue",
        'color': 'lightskyblue',
        'fillOpacity':0.7
    },
    name= lgd_txt.format(txt='White Population', col= 'lightskyblue')
).add_to(chicago_map)

# Plotting the white population
baaa = list(zip_ethnicity[zip_ethnicity['ethnicity'] == 'Black or African American Alone']['zip'])
baaa = zipGeom[zipGeom['zip'].isin(baaa)]
folium.GeoJson(
    baaa.to_json(),
    style_function=lambda feature: {
        'fillColor': "dodgerblue",
        'color': 'dodgerblue',
        'opacity':0.7,
        'fillOpacity':0.7
    },
    name= lgd_txt.format(txt='Black or African American Population', col= 'dodgerblue')
).add_to(chicago_map)

# Plotting the white population
hol = list(zip_ethnicity[zip_ethnicity['ethnicity'] == 'Hispanic or Latino']['zip'])
hol = zipGeom[zipGeom['zip'].isin(hol)]
folium.GeoJson(
    hol.to_json(),
    style_function=lambda feature: {
        'fillColor': "lightgreen",
        'color': 'lightgreen',
        'opacity':0.7,
        'fillOpacity':0.7
    },
    name= lgd_txt.format(txt='Hispanic or Latino', col= 'lightgreen')
).add_to(chicago_map)



# Inspected Restaurants
usZipLL = pd.read_csv('./data/uszipcodelatlong.csv')
chicago_inspections["zip"] = chicago_inspections['zip'].astype(int)
zipGeom = gpd.read_file("./data/zipCodesBoundaries.geojson")
zipGeom['zip'] = zipGeom['zip'].astype(int)
zipGeom['shape_area'] = zipGeom['shape_area'].astype(float)
bubbles = chicago_inspections.groupby('zip').nunique()[['license']].reset_index()

# Plotting
fg = FeatureGroup(name='#Restaurants').add_to(chicago_map)
#restaurants = plugins.FeatureGroupSubGroup(fg, 'Restaurants')
for i in range(0, len(bubbles)):
    try:
        zipcode = bubbles.iloc[i]['zip']
        zipinfo = usZipLL[usZipLL['ZIP'] == zipcode]
        area = zipGeom[zipGeom['zip'] == zipcode]['shape_area']
        if len(area) == 0:
            continue
        area = area * 10 / np.mean(zipGeom['shape_area'])
        area = area.values[0]
        location = [zipinfo['LAT'], zipinfo['LNG']]
        name = str(bubbles.iloc[i]['zip'])
        folium.Circle(
                    location=location,
                    popup=name,
                    name='restaurant',
                    radius = bubbles.iloc[i]['license']/area,
                    color='black',
                    fill=True,
                    fill_color='black',
                    tooltip = str(bubbles.iloc[i]['license']) + ' restaurants'
                ).add_to(fg)
    except TypeError:
        ''
    except ValueError:
        ''
        
        
#chicago_map.add_child(restaurants)  
    
folium.LayerControl( collapsed=False).add_to(chicago_map)

chicago_map.save('./maps/DistributionEthnicityRestaurants.html')

In [79]:
#__________________ Amount of Restaurants and Inspections Per Majority __________________#
# Data for the amount of restaurants
data = chicago_inspections.groupby('majority_ethnicity').nunique()[['license']].reset_index()
data = data[data['majority_ethnicity'].isin(ethnicities)]
data.columns = ["ethnicity", "amount of restaurants"]
data['color'] = ['dodgerblue', 'lightgreen', 'lightskyblue']
data[y] = data[y] * 100 / len(chicago_inspections['license'].unique())
hovertemplate1 = '%{y:.2f}% of the restaurants are in neighbourhoods dominated by %{x}'

# Data for the amount of inspections
temp = chicago_inspections.groupby('majority_ethnicity').count()[['license']].reset_index()
temp = temp[temp['majority_ethnicity'].isin(ethnicities)]
temp.columns = ["ethnicity", "amount of restaurants"]
temp['color'] = ['dodgerblue', 'lightgreen', 'lightskyblue']
x = "ethnicity"
y = "amount of restaurants"
temp[y] = temp[y] * 100 / len(chicago_inspections)
data["inspections"] = temp[y]
hovertemplate2 = '%{y:.2f}% of the inspections have been in neighbourhoods dominated by %{x}'

# plot
title = "Distribution of the Amount of Restaurants and Inspections Conducted Accross Each Ethnic Areas"
fig = go.Figure()
data= resto_distribution.sort_values('amount of restaurants', ascending=False)
fig.add_trace(go.Bar(
    x=data['ethnicity'],
    y=data['amount of restaurants'],
    opacity=0.8, 
    name='Amount of restaurants',
    hovertemplate = hovertemplate1
))
fig.add_trace(go.Bar(
    x=data['ethnicity'],
    y=data['inspections'],
    opacity=0.4, 
    name='Amount of inspections',
    hovertemplate = hovertemplate2
))

fig.update_traces(marker_color=data['color'], 
                  marker_line_width=1.5, 
                  )
fig.update_layout(
     title={
        'text': title,
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}
    )
    
fig.show()

In [179]:
zipchic

,ethnicity,shape_area,area to explore to discover a restaurant
0,Black or African American Alone,2.494064e+09,295890.875570
1,Hispanic or Latino,1.679143e+09,242475.456815
2,White Alone,2.270599e+09,103713.486435


In [180]:
#__________________ Area to explore before finding a restaurant __________________#
# Creating data
zipchic = zipGeom[zipGeom['zip'].isin(chicago_inspections["zip"].unique())]
zip_ethnicity['zip'] = zip_ethnicity['zip'].astype(int)
zipchic = zipchic.merge(zip_ethnicity, on=['zip'], how='left')
zipchic = zipchic.groupby('ethnicity').sum()[['shape_area']].reset_index()

temp = chicago_inspections.groupby('majority_ethnicity').nunique()[['license']].reset_index()
temp = temp[temp['majority_ethnicity'].isin(ethnicities)].reset_index(drop=True)

zipchic['area to explore to discover a restaurant'] = zipchic['shape_area'] / temp['license']
zipchic['color'] = ['dodgerblue', 'lightgreen', 'lightskyblue']
zipchic['arbitrary_order'] = [2, 3, 1]
zipchic.sort_values('arbitrary_order')
x = 'ethnicity'
y = 'area to explore to discover a restaurant'
title = 'Area to explore to discover at least one restaurant'
hovertemplate = 'You need to explore %{y:.2f} to make sure you discover one restaurant'
plot_bardata(zipchic, x, y, title, hovertemplate, order={'ethnicity':ethnicities})

In [188]:
#__________________ Distribution of Pass/Fail/Out of Business __________________#
chicago_inspections['pass'] = chicago_inspections['results'] == 'Pass'
chicago_inspections['out of business'] = chicago_inspections['results'] == 'Out of Business'
chicago_inspections['fail'] = chicago_inspections['results'] == 'Fail'

chicago_inspections['pass'] = chicago_inspections['pass'].astype(int)
chicago_inspections['out of business'] = chicago_inspections['out of business'].astype(int)
chicago_inspections['fail'] = chicago_inspections['fail'].astype(int)

data = chicago_inspections.groupby('majority_ethnicity').sum()[['pass', 'out of business', 'fail']].reset_index()
data = data[data['majority_ethnicity'].isin(ethnicities)]

data['total'] = np.sum(data, axis=1)
data['pass'] = data['pass'] * 100 / data['total']
data['out of business'] = data['out of business'] * 100 / data['total']
data['fail'] = data['fail'] * 100 / data['total']

hovertemplate1 = '%{y:.2f}% passed in neighbourhoods dominated by %{x}'
hovertemplate2 = '%{y:.2f}% failed in neighbourhoods dominated by %{x}'
hovertemplate3 = '%{y:.2f}% went out of business in neighbourhoods dominated by %{x}'

# plot
title = "Distribution of Pass/Fail/Out of Business per Ethnic Areas"
fig = go.Figure()
data = data.sort_values('total', ascending=False)
data['color'] = ['lightskyblue', 'dodgerblue', 'lightgreen']
fig.add_trace(go.Bar(
    x=data['majority_ethnicity'],
    y=data['pass'],
    opacity=0.9, 
    name='Passing rate',
    hovertemplate = hovertemplate1
))
fig.add_trace(go.Bar(
    x=data['majority_ethnicity'],
    y=data['fail'],
    opacity=0.6, 
    name='Failing rate',
    hovertemplate = hovertemplate2
))

fig.add_trace(go.Bar(
    x=data['majority_ethnicity'],
    y=data['out of business'],
    opacity=0.2, 
    name='Out of business rate',
    hovertemplate = hovertemplate3
))


fig.update_traces(marker_color=data['color'], 
                  marker_line_width=1.5, 
                  )
fig.update_layout(
     title={
        'text': title,
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}
    )
    
fig.show()

In [280]:
chicago_inspections['whites'] = chicago_inspections['majority_ethnicity'] == ethnicities[0]
chicago_inspections['baaa'] = chicago_inspections['majority_ethnicity'] == ethnicities[1]
chicago_inspections['hol'] = chicago_inspections['majority_ethnicity'] == ethnicities[2]

chicago_inspections['whites'] = chicago_inspections['whites'].astype(float)
chicago_inspections['baaa'] = chicago_inspections['baaa'].astype(float)
chicago_inspections['hol'] = chicago_inspections['hol'].astype(float)

whites = len(chicago_inspections[chicago_inspections['majority_ethnicity'] == ethnicities[0]])
baaa = len(chicago_inspections[chicago_inspections['majority_ethnicity'] == ethnicities[1]])
hol = len(chicago_inspections[chicago_inspections['majority_ethnicity'] == ethnicities[2]])

data = chicago_inspections.groupby('results').sum()[['whites', 'baaa', 'hol']].reset_index()
data['whites'] /= whites
data['baaa'] /= baaa
data['hol'] /= hol

data['whites'] *= 100
data['baaa'] *= 100
data['hol'] *= 100

In [281]:
data

,results,whites,baaa,hol
0,Business Not Located,0.025491,0.059378,0.028505
1,Fail,18.053361,22.632592,19.713813
2,No Entry,3.551704,2.375128,2.907474
3,Not Ready,0.993287,0.883547,0.917850
4,Out of Business,8.294672,9.134741,9.013169
5,Pass,54.225508,53.449873,54.580697
6,Pass w/ Conditions,14.855978,11.464741,12.838493


In [283]:
title = "Distribution of the Results for each Majority "
fig = go.Figure()
hovertemplate1 = '%{y:.2f}% passed in neighbourhoods dominated by ' + ethnicities[0]
fig.add_trace(go.Bar(
    x=data['results'],
    y=data['whites'],
    opacity=0.6, 
    name=ethnicities[0],
    hovertemplate = hovertemplate1,
    marker_color='lightskyblue', 
))
hovertemplate2 = '%{y:.2f}% passed in neighbourhoods dominated by ' + ethnicities[1]
fig.add_trace(go.Bar(
    x=data['results'],
    y=data['baaa'],
    opacity=0.6, 
    name=ethnicities[1],
    hovertemplate = hovertemplate2,
    marker_color='dodgerblue', 
))

hovertemplate3 = '%{y:.2f}% passed in neighbourhoods dominated by ' + ethnicities[2]
fig.add_trace(go.Bar(
    x=data['results'],
    y=data['hol'],
    opacity=0.6, 
    name=ethnicities[2],
    hovertemplate = hovertemplate3,
    marker_color='lightgreen', 
))


fig.update_traces(
                  marker_line_width=1.5, 
                  )
fig.update_layout(
     title={
        'text': title,
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}
    )
    
fig.show()

In [204]:
chicago_inspections.head(1)

,inspection_id,dba_name,aka_name,license,facility_type,risk,address,city,state,zip,...,location,new_facility_type,new_violations,majority_ethnicity,pass,out of business,fail,whites,baaa,hol
0,2320977,todd appel bar and cocktail,todd appel bar and cocktail,2589615.0,grocery store,Risk 2 (Medium),5868 N LINCOLN AVE,chicago,IL,60659,...,"{'latitude': '-87.70370445181302', 'longitude'...",grocery store,[],White Alone,0,0,0,1.0,0.0,0.0


In [205]:
tract.head(1)

,zip,tract,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,37801,47009011603,0.034972,0.042088,0.0,0.035515


In [207]:
ethnicity_df.head(1)

,GEONAME,lntitle,geoid,lnnumber,CIT_EST,CIT_MOE,CVAP_EST,CVAP_MOE,onmap,tract_latitude,tract_longitude,geoid_match
274274,"Census Tract 101, Cook County, Illinois",Total,14000US17031010100,1,4015,358,3200,319,1,379751.0,0.0,17031010100


In [211]:
print(chicago_inspections['zip'].unique())

[60659 60707 60646 60611 60630 60634 60643 60638 60618 60625 60639 60647
 60619 60623 60666 60607 60629 60606 60641 60632 60609 60608 60654 60601
 60613 60605 60614 60620 60657 60624 60651 60636 60622 60621 60645 60653
 60656 60655 60640 60628 60616 60644 60603 60612 60642 60649 60652 60626
 60660 60602 60661 60637 60631 60604 60615 60610 60617 60633 60805 60827
 60804 60455 60302 60153 60627 60402 60453 60155]


In [224]:
zip_pop = pd.read_excel('./data/zip_population.xlsx', index=None, header=None)
zip_pop.columns = ['zip', 'population']

In [225]:
zip_pop.head()

,zip,population
0,60659,39155
1,60707,42621
2,60646,27016
3,60611,26522
4,60630,54781


In [ ]:
#__________________ Area to explore before finding a restaurant __________________#
# Creating data
zipchic = zipGeom[zipGeom['zip'].isin(chicago_inspections["zip"].unique())]
zip_ethnicity['zip'] = zip_ethnicity['zip'].astype(int)
zipchic = zipchic.merge(zip_ethnicity, on=['zip'], how='left')
zipchic = zipchic.groupby('ethnicity').sum()[['shape_area']].reset_index()

temp = chicago_inspections.groupby('majority_ethnicity').nunique()[['license']].reset_index()
temp = temp[temp['majority_ethnicity'].isin(ethnicities)].reset_index(drop=True)

zipchic['area to explore to discover a restaurant'] = zipchic['shape_area'] / temp['license']
zipchic['color'] = ['dodgerblue', 'lightgreen', 'lightskyblue']
zipchic['arbitrary_order'] = [2, 3, 1]
zipchic.sort_values('arbitrary_order')
x = 'ethnicity'
y = 'area to explore to discover a restaurant'
title = 'Area to explore to discover at least one restaurant'
hovertemplate = 'You need to explore %{y:.2f} to make sure you discover one restaurant'
plot_bardata(zipchic, x, y, title, hovertemplate, order={'ethnicity':ethnicities})

In [220]:
chicago_inspections.head(1)

,inspection_id,dba_name,aka_name,license,facility_type,risk,address,city,state,zip,...,location,new_facility_type,new_violations,majority_ethnicity,pass,out of business,fail,whites,baaa,hol
0,2320977,todd appel bar and cocktail,todd appel bar and cocktail,2589615.0,grocery store,Risk 2 (Medium),5868 N LINCOLN AVE,chicago,IL,60659,...,"{'latitude': '-87.70370445181302', 'longitude'...",grocery store,[],White Alone,0,0,0,1.0,0.0,0.0


In [230]:
ethnicity_df

,GEONAME,lntitle,geoid,lnnumber,CIT_EST,CIT_MOE,CVAP_EST,CVAP_MOE,onmap,tract_latitude,tract_longitude,geoid_match
274274,"Census Tract 101, Cook County, Illinois",Total,14000US17031010100,1,4015,358,3200,319,1,379751.0,0.0,17031010100
274275,"Census Tract 101, Cook County, Illinois",Not Hispanic or Latino,14000US17031010100,2,3655,408,2940,344,1,379751.0,0.0,17031010100
274276,"Census Tract 101, Cook County, Illinois",American Indian or Alaska Native Alone,14000US17031010100,3,0,11,0,11,1,379751.0,0.0,17031010100
274277,"Census Tract 101, Cook County, Illinois",Asian Alone,14000US17031010100,4,4,11,4,11,1,379751.0,0.0,17031010100
274278,"Census Tract 101, Cook County, Illinois",Black or African American Alone,14000US17031010100,5,1890,472,1300,357,1,379751.0,0.0,17031010100
...,...,...,...,...,...,...,...,...,...,...,...,...
291403,"Census Tract 9801, Cook County, Illinois",Asian and White,14000US17031980100,9,0,11,0,11,1,2981781.0,0.0,17031980100
291404,"Census Tract 9801, Cook County, Illinois",Black or African American and White,14000US17031980100,10,0,11,0,11,1,2981781.0,0.0,17031980100
291405,"Census Tract 9801, Cook County, Illinois",American Indian or Alaska Native and Black or ...,14000US17031980100,11,0,11,0,11,1,2981781.0,0.0,17031980100
291406,"Census Tract 9801, Cook County, Illinois",Remainder of Two or More Race Responses,14000US17031980100,12,0,11,0,11,1,2981781.0,0.0,17031980100


In [238]:
data = zip_ethnicity.merge(zip_pop, on=['zip'], how='left')
data = data.groupby('ethnicity').sum()[['population']].reset_index()

In [255]:
n_restaurants = chicago_inspections.groupby('majority_ethnicity').nunique()[['license']].reset_index()
n_restaurants = n_restaurants[n_restaurants['majority_ethnicity'].isin(ethnicities)].reset_index(drop=True)

In [256]:
n_restaurants

,majority_ethnicity,license
0,Black or African American Alone,8429
1,Hispanic or Latino,6925
2,White Alone,21893


In [251]:
data

,ethnicity,population,habitants per restaurants
0,Black or African American Alone,1015606.0,NaN
1,Hispanic or Latino,807946.0,95.853126
2,White Alone,1495646.0,215.977762


In [252]:
n_restaurants

,majority_ethnicity,license
1,Black or African American Alone,8429.0
2,Hispanic or Latino,6925.0
3,White Alone,21893.0


In [257]:
data['population'] = data['population'].astype(float)
n_restaurants['license'] = n_restaurants['license'].astype(float)
data['habitants per restaurants'] = data['population'] / n_restaurants['license']

In [258]:
data

,ethnicity,population,habitants per restaurants
0,Black or African American Alone,1015606.0,120.489501
1,Hispanic or Latino,807946.0,116.670903
2,White Alone,1495646.0,68.316174


In [261]:
#__________________ Area to explore before finding a restaurant __________________#
# Creating data
x = 'ethnicity'
y = 'habitants per restaurants'
title = 'Habitants per restaurants'
hovertemplate = '%{y:.2f} habitants per restaurants'
data['color'] = ['dodgerblue', 'lightskyblue', 'lightgreen']
plot_bardata(data, x, y, title, hovertemplate, order={'ethnicity':ethnicities})

In [ ]:
fig.add_trace(go.Bar(
    x=data['results'],
    y=data['whites'],
    opacity=0.6, 
    name=ethnicities[0],
    hovertemplate = hovertemplate1,
    marker_color='lightskyblue', 
))
hovertemplate2 = '%{y:.2f}% passed in neighbourhoods dominated by ' + ethnicities[1]
fig.add_trace(go.Bar(
    x=data['results'],
    y=data['baaa'],
    opacity=0.6, 
    name=ethnicities[1],
    hovertemplate = hovertemplate2,
    marker_color='dodgerblue', 
))

hovertemplate3 = '%{y:.2f}% passed in neighbourhoods dominated by ' + ethnicities[2]
fig.add_trace(go.Bar(
    x=data['results'],
    y=data['hol'],
    opacity=0.6, 
    name=ethnicities[2],
    hovertemplate = hovertemplate3,
    marker_color='lightgreen', 
))

In [264]:
data

,ethnicity,population,habitants per restaurants,color
0,Black or African American Alone,1015606.0,120.489501,dodgerblue
1,Hispanic or Latino,807946.0,116.670903,lightskyblue
2,White Alone,1495646.0,68.316174,lightgreen


In [276]:
x = list(range(400))
y0 = x + n_restaurants['license'].iloc[0]
y0 = data['population'].iloc[0] / y0
y1 = x + n_restaurants['license'].iloc[1]
y1 = data['population'].iloc[1] / y1
y2 = x + n_restaurants['license'].iloc[2]
y3 = data['population'].iloc[2] / y2